In [1]:
import os
import math
import torch
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import models
from sklearn.model_selection import KFold, train_test_split
from scipy.stats import spearmanr, pearsonr
import numpy as np
from sklearn.preprocessing import normalize
from einops import rearrange, repeat

# from datasets.AI1k import AI1k_load_data
# from utils.compute_metrics import compute_metrics
from utils.finetune_model_Jean import finetune_model
from models.MetaIQA import MetaIQA
from models.CNNIQAnet import CNNIQAnet
#from models.BRISQUE import BRISQUE

num_epochs = 25
num_splits = 10

/home/ccl/anaconda3/envs/MetaIQA/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
exp_name = "CHAT_Image_features_BRISQUE_LIVE"

class FusionModel(nn.Module):
    def __init__(self):
        super(FusionModel, self).__init__()
        
        # Load the pretrained ResNet18 model
        self.quality_model = CNNIQAnet()
        # self.quality_model = MetaIQA('model_IQA/TID2013_KADID10K_IQA_Meta_resnet18.pt')
        input_size = 2084
        intermediate_size = 512

        self.fc_clip_feat = nn.Linear(512, 512)
     
        self.relu_clip_feat = nn.PReLU()
        self.drop_clip_feat = nn.Dropout(p=0.5)
        

        self.fc_hps_feat = nn.Linear(768, 768)
     
        self.relu_hps_feat = nn.PReLU()
        self.drop_hps_feat = nn.Dropout(p=0.5)
        

        self.fc_alt_feat = nn.Linear(768, 768)
     
        self.relu_alt_feat = nn.PReLU()
        self.drop_alt_feat = nn.Dropout(p=0.5)

        self.fc_final = nn.Linear(2048, 2048)
        
        
        # # Define the fully connected layer
        self.fc1 = nn.Linear(input_size, input_size)
    
        self.fc2 = nn.Linear(input_size, intermediate_size)

        self.fc3 = nn.Linear(intermediate_size, 1)
        self.sig = nn.Sigmoid()


        
    def forward(self, x, brisque_features, similarity_features, similarity_features1, similarity_features2):
        # Pass the input through the pretrained model 
        quality, quality_features = self.quality_model(x)

        similarity_features = self.fc_clip_feat(similarity_features)
        similarity_features = self.relu_clip_feat(similarity_features)
        similarity_features = self.drop_clip_feat(similarity_features)

        similarity_features1 = self.fc_hps_feat(similarity_features1)
        similarity_features1 = self.relu_hps_feat(similarity_features1)
        similarity_features1 = self.drop_hps_feat(similarity_features1)

        similarity_features2 = self.fc_alt_feat(similarity_features2)
        similarity_features2 = self.relu_alt_feat(similarity_features2)
        similarity_features2 = self.drop_alt_feat(similarity_features2)

        combined = torch.cat((similarity_features, similarity_features1, similarity_features2), dim=1)

        final_fc = self.fc_final(combined)

        # Concatenate the final prediction output and the new input
        combined = torch.cat((brisque_features, final_fc), dim=1)
        
        x = self.fc1(combined)
        x = self.fc2(x)
        x = self.fc3(x)
        return self.sig(x)
        # return x
    
def fetch_example(data):
    inputs = data['image']
    similarity_features = data['clip_image_features']
    similarity_features1 = data['clip_image_features1']
    similarity_features2 = data['clip_image_features2']
    brisque_features = data['brisque_features']
    batch_size = inputs.size()[0]
    labels = data['rating'].view(batch_size, -1) / 100.0
    
    inputs, brisque_features, similarity_features, similarity_features1, similarity_features2, labels = Variable(inputs.float().cuda()), Variable(brisque_features.float().cuda()), Variable(similarity_features.float().cuda()), Variable(similarity_features1.float().cuda()), Variable(similarity_features2.float().cuda()), Variable(labels.float().cuda())
    return [inputs, brisque_features, similarity_features, similarity_features1, similarity_features2], labels

finetune_model(FusionModel, fetch_example, exp_name, num_epochs = num_epochs, num_splits = num_splits)

decay_rate is set to 1.0
Train Results - Split: 0, Epoch: 0, PLCC: 0.6784, SROCC: 0.6708, MSE: 0.0314
Validation Results - Split: 0, Epoch: 0, PLCC: 0.6323, SROCC: 0.6467, MSE: 0.0340, Best PLCC: 0.6323, Best SROCC: 0.6467, Best MSE: 0.0340
Train Results - Split: 0, Epoch: 1, PLCC: 0.7885, SROCC: 0.7668, MSE: 0.0165
Validation Results - Split: 0, Epoch: 1, PLCC: 0.7434, SROCC: 0.7385, MSE: 0.0202, Best PLCC: 0.7434, Best SROCC: 0.7385, Best MSE: 0.0202
Train Results - Split: 0, Epoch: 2, PLCC: 0.8583, SROCC: 0.8274, MSE: 0.0133
Validation Results - Split: 0, Epoch: 2, PLCC: 0.7980, SROCC: 0.7833, MSE: 0.0186, Best PLCC: 0.7980, Best SROCC: 0.7833, Best MSE: 0.0186
Train Results - Split: 0, Epoch: 3, PLCC: 0.8725, SROCC: 0.8471, MSE: 0.0130
Validation Results - Split: 0, Epoch: 3, PLCC: 0.8065, SROCC: 0.7880, MSE: 0.0192, Best PLCC: 0.8065, Best SROCC: 0.7880, Best MSE: 0.0186
Train Results - Split: 0, Epoch: 4, PLCC: 0.8955, SROCC: 0.8707, MSE: 0.0086
Validation Results - Split: 0, Epo

: 

In [2]:
exp_name = "CLIP_features_2fc_Meta_A3k-(ViT-16)-ComCLIP-Morph"

class FusionModel(nn.Module):
    def __init__(self):
        super(FusionModel, self).__init__()
        
        # Load the pretrained ResNet18 model
        self.quality_model = MetaIQA('model_IQA/TID2013_KADID10K_IQA_Meta_resnet18.pt')
        input_size = 1536

        intermediate_size = 512

        self.fc_clip_image = nn.Linear(512, 512)
        self.fc_clip_text = nn.Linear(512, 512)
        self.fc_clip_text.weight.data.normal_(0, 0.0)
        self.fc_clip_text.bias.data.zero_()
        
        self.relu_clip_feat = nn.PReLU()
        self.drop_clip_feat = nn.Dropout(p=0.5)

        self.fc_final = nn.Linear(1024, 1024)
        # self.sig = nn.Sigmoid()
        # self.attention = nn.MultiheadAttention(embed_dim=input_size, num_heads=1)

        # Define the fully connected layers
        self.fc1 = nn.Linear(input_size, input_size)
        self.fc2 = nn.Linear(input_size, intermediate_size)
        self.fc3 = nn.Linear(intermediate_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, clip_image, clip_text):
        # Pass the input through the pretrained model
        quality, quality_features = self.quality_model(x)

        image_out = self.fc_clip_image(clip_image)
        # # similarity_features = self.bn_clip_feat(similarity_features)
        image_out = self.relu_clip_feat(image_out)
        image_out = self.drop_clip_feat(image_out)

        text_out = self.fc_clip_text(clip_text)
        # # similarity_features = self.bn_clip_feat(similarity_features)
        text_out = self.relu_clip_feat(text_out)
        text_out = self.drop_clip_feat(text_out)

        combined = torch.cat((image_out, text_out), dim=1)
        fc_final = self.fc_final(combined)

        combined_1 = torch.cat((quality_features, fc_final), dim=1)


        
        # Pass the concatenated tensor through the fully connected layer
        x = self.fc1(combined_1)
        x = self.fc2(x)
        x = self.fc3(x)
        return self.sigmoid(x)
        # return x
    
def fetch_example(data):
    inputs = data['image']
    clip_image = data['clip_image_features']
    clip_text = data['clip_text_features']
    batch_size = inputs.size()[0]
    labels = data['rating'].view(batch_size, -1) / 5.0
    
    inputs, clip_image, clip_text, labels = Variable(inputs.float().cuda()), Variable(clip_image.float().cuda()),Variable(clip_text.float().cuda()), Variable(labels.float().cuda())
    return [inputs, clip_image, clip_text], labels

finetune_model(FusionModel, fetch_example, exp_name, num_epochs = num_epochs, num_splits = num_splits)


decay_rate is set to 1.0
Train Results - Split: 0, Epoch: 0, PLCC: 0.9064, SROCC: 0.8593, MSE: 0.0071
Validation Results - Split: 0, Epoch: 0, PLCC: 0.8792, SROCC: 0.8197, MSE: 0.0093, Best PLCC: 0.8792, Best SROCC: 0.8197, Best MSE: 0.0093
Train Results - Split: 0, Epoch: 1, PLCC: 0.9324, SROCC: 0.8961, MSE: 0.0056
Validation Results - Split: 0, Epoch: 1, PLCC: 0.8755, SROCC: 0.8195, MSE: 0.0096, Best PLCC: 0.8792, Best SROCC: 0.8197, Best MSE: 0.0093
Train Results - Split: 0, Epoch: 2, PLCC: 0.9366, SROCC: 0.9147, MSE: 0.0053
Validation Results - Split: 0, Epoch: 2, PLCC: 0.8692, SROCC: 0.8227, MSE: 0.0101, Best PLCC: 0.8792, Best SROCC: 0.8227, Best MSE: 0.0093
Train Results - Split: 0, Epoch: 3, PLCC: 0.9405, SROCC: 0.9288, MSE: 0.0075
Validation Results - Split: 0, Epoch: 3, PLCC: 0.8680, SROCC: 0.8290, MSE: 0.0145, Best PLCC: 0.8792, Best SROCC: 0.8290, Best MSE: 0.0093
Train Results - Split: 0, Epoch: 4, PLCC: 0.9485, SROCC: 0.9338, MSE: 0.0048
Validation Results - Split: 0, Epo

In [4]:
exp_name = "CLIP_features_diff_Meta-AI-3k_(ViT-16)-MorphemePrompt_ComCLIP"

class FusionModel(nn.Module):
    def __init__(self):
        super(FusionModel, self).__init__()
        
        # Load the pretrained ResNet18 model
        self.quality_model = MetaIQA('model_IQA/TID2013_KADID10K_IQA_Meta_resnet18.pt')
        input_size = 1024
        intermediate_size = 512

        self.fc_clip_feat = nn.Linear(512, 512)
        # self.fc_clip_feat.weight.data.normal_(0, 0.0)
        # self.fc_clip_feat.bias.data.zero_()
        # self.bn_clip_feat = nn.BatchNorm1d(512)
        self.relu_clip_feat = nn.PReLU()
        self.drop_clip_feat = nn.Dropout(p=0.5)
        self.fc_final = nn.Linear(512,512)
        
        # # Define the fully connected layer
        self.fc1 = nn.Linear(input_size, input_size)
        self.fc2 = nn.Linear(input_size, intermediate_size)
        self.fc3 = nn.Linear(intermediate_size, 1)
        self.sigmoid = nn.Sigmoid()

        
    def forward(self, x, similarity_features):
        # Pass the input through the pretrained model 
        quality, quality_features = self.quality_model(x)
        # print (quality_features.shape)

        similarity_features = self.fc_clip_feat(similarity_features)
        # similarity_features = self.bn_clip_feat(similarity_features)
        similarity_features = self.relu_clip_feat(similarity_features)
        similarity_features = self.drop_clip_feat(similarity_features)

        final = self.fc_final(similarity_features)
        # Concatenate the final prediction output and the new input
        combined = torch.cat((quality_features, final), dim=1)

        x = self.fc1(combined)
        x = self.fc2(x)
        x = self.fc3(x)
        return self.sigmoid(x)
        # return x
    
def fetch_example(data):
    inputs = data['image']
    similarity_features = data['clip_image_features'] - data['clip_text_features']
    batch_size = inputs.size()[0]
    labels = data['rating'].view(batch_size, -1) / 5.0
    
    inputs, similarity_features, labels = Variable(inputs.float().cuda()), Variable(similarity_features.float().cuda()), Variable(labels.float().cuda())
    return [inputs, similarity_features], labels

finetune_model(FusionModel, fetch_example, exp_name, num_epochs = num_epochs, num_splits = num_splits)

decay_rate is set to 1.0
Train Results - Split: 0, Epoch: 0, PLCC: 0.9258, SROCC: 0.8879, MSE: 0.0059
Validation Results - Split: 0, Epoch: 0, PLCC: 0.9045, SROCC: 0.8663, MSE: 0.0078, Best PLCC: 0.9045, Best SROCC: 0.8663, Best MSE: 0.0078
Train Results - Split: 0, Epoch: 1, PLCC: 0.9530, SROCC: 0.9276, MSE: 0.0038
Validation Results - Split: 0, Epoch: 1, PLCC: 0.9115, SROCC: 0.8809, MSE: 0.0073, Best PLCC: 0.9115, Best SROCC: 0.8809, Best MSE: 0.0073
Train Results - Split: 0, Epoch: 2, PLCC: 0.9662, SROCC: 0.9499, MSE: 0.0030
Validation Results - Split: 0, Epoch: 2, PLCC: 0.9063, SROCC: 0.8738, MSE: 0.0081, Best PLCC: 0.9115, Best SROCC: 0.8809, Best MSE: 0.0073
Train Results - Split: 0, Epoch: 3, PLCC: 0.9653, SROCC: 0.9512, MSE: 0.0030
Validation Results - Split: 0, Epoch: 3, PLCC: 0.8973, SROCC: 0.8578, MSE: 0.0087, Best PLCC: 0.9115, Best SROCC: 0.8809, Best MSE: 0.0073
Train Results - Split: 0, Epoch: 4, PLCC: 0.9725, SROCC: 0.9595, MSE: 0.0024
Validation Results - Split: 0, Epo

: 

In [2]:
exp_name = "CLIP_Image_Meta_IQA-AGIQA3k-November-ADAIN"


def calc_mean_std(features):
    # Calculate mean and std for the feature tensor along feature dimension only
    feature_mean = features.mean(dim=1, keepdim=True)
    feature_std = features.std(dim=1, keepdim=True) + 1e-6
    return feature_mean, feature_std

def adain(content_features, style_features):
    content_mean, content_std = calc_mean_std(content_features)
    style_mean, style_std = calc_mean_std(style_features)
    normalized_features = style_std * (content_features - content_mean) / content_std + style_mean
    # print(normalized_features.shape)
    return normalized_features


class FusionModel(nn.Module):
    def __init__(self):
        super(FusionModel, self).__init__()
        
        # Load the pretrained ResNet18 model
        self.quality_model = MetaIQA('model_IQA/TID2013_KADID10K_IQA_Meta_resnet18.pt')
        # self.quality_model = CNNIQAnet()
        input_size = 1024
        intermediate_size = 512

        self.fc_clip_feat = nn.Linear(512, 512)
        self.fc_clip_feat.weight.data.normal_(0, 0.0)
        self.fc_clip_feat.bias.data.zero_()
        self.relu_clip_feat = nn.PReLU()
        self.drop_clip_feat = nn.Dropout(p=0.5)
        self.fc_final = nn.Linear(512, 512)
        
        # # Define the fully connected layer
        # self.fc1 = nn.Linear(input_size, input_size)
    
        # self.fc2 = nn.Linear(input_size, intermediate_size)

        self.fc3 = nn.Linear(intermediate_size, 1)
        self.sig = nn.Sigmoid()


        
    def forward(self, x, similarity_features):
        # Pass the input through the pretrained model 
        quality, quality_features = self.quality_model(x)

        # Process quality and similarity features
        similarity_features = self.fc_clip_feat(similarity_features)
        similarity_features = self.relu_clip_feat(similarity_features)
        similarity_features = self.drop_clip_feat(similarity_features)

        # Pass the similarity_features through fc final
        simi_features = self.fc_final(similarity_features)

        # Compute mean and std for quality features
        quality_mean, quality_std = calc_mean_std(quality_features)

        # Compute mean and std for similarity features
        similarity_mean, similarity_std = calc_mean_std(simi_features)

        # Normalize the quality features using AdaIN
        normalized_quality_features = adain(simi_features, quality_features)
        # print(normalized_quality_features.shape)

        # Pass the normalized quality features through the fully connected layers
        # x = self.fc1(x)
        # x = self.fc2(x)
        x = self.fc3(normalized_quality_features)
        return self.sig(x)
        # return x
    
def fetch_example(data):
    inputs = data['image']
    similarity_features = data['clip_image_features']- data['clip_text_features']
    batch_size = inputs.size()[0]
    labels = data['rating'].view(batch_size, -1) / 5.0
    
    inputs, similarity_features, labels = Variable(inputs.float().cuda()), Variable(similarity_features.float().cuda()), Variable(labels.float().cuda())
    return [inputs, similarity_features], labels

finetune_model(FusionModel, fetch_example, exp_name, num_epochs = num_epochs, num_splits = num_splits)

decay_rate is set to 1.0
Train Results - Split: 0, Epoch: 0, PLCC: 0.9198, SROCC: 0.8793, MSE: 0.0062
Validation Results - Split: 0, Epoch: 0, PLCC: 0.9048, SROCC: 0.8637, MSE: 0.0074, Best PLCC: 0.9048, Best SROCC: 0.8637, Best MSE: 0.0074
Train Results - Split: 0, Epoch: 1, PLCC: 0.9426, SROCC: 0.9153, MSE: 0.0056
Validation Results - Split: 0, Epoch: 1, PLCC: 0.9064, SROCC: 0.8642, MSE: 0.0082, Best PLCC: 0.9064, Best SROCC: 0.8642, Best MSE: 0.0074
Train Results - Split: 0, Epoch: 2, PLCC: 0.9621, SROCC: 0.9504, MSE: 0.0045
Validation Results - Split: 0, Epoch: 2, PLCC: 0.9040, SROCC: 0.8632, MSE: 0.0087, Best PLCC: 0.9064, Best SROCC: 0.8642, Best MSE: 0.0074
Train Results - Split: 0, Epoch: 3, PLCC: 0.9688, SROCC: 0.9602, MSE: 0.0031
Validation Results - Split: 0, Epoch: 3, PLCC: 0.9028, SROCC: 0.8604, MSE: 0.0080, Best PLCC: 0.9064, Best SROCC: 0.8642, Best MSE: 0.0074
Train Results - Split: 0, Epoch: 4, PLCC: 0.9668, SROCC: 0.9583, MSE: 0.0034
Validation Results - Split: 0, Epo

In [2]:
exp_name = "CLIP_Image_Meta_IQA-LIVE-October_16"


def calc_mean_std(features):
    # Calculate mean and std for the feature tensor along feature dimension only
    feature_mean = features.mean(dim=1, keepdim=True)
    feature_std = features.std(dim=1, keepdim=True) + 1e-6
    return feature_mean, feature_std

def adain(content_features, style_features):
    content_mean, content_std = calc_mean_std(content_features)
    style_mean, style_std = calc_mean_std(style_features)
    normalized_features = style_std * (content_features - content_mean) / content_std + style_mean
    print(normalized_features.shape)
    return normalized_features


class FusionModel(nn.Module):
    def __init__(self):
        super(FusionModel, self).__init__()
        
        # Load the pretrained ResNet18 model
        self.quality_model = MetaIQA('model_IQA/TID2013_KADID10K_IQA_Meta_resnet18.pt')
        # self.quality_model = CNNIQAnet()
        input_size = 1024
        intermediate_size = 512

        self.fc_clip_feat = nn.Linear(512, 512)
     
        self.relu_clip_feat = nn.PReLU()
        self.drop_clip_feat = nn.Dropout(p=0.5)
        self.fc_final = nn.Linear(512, 512)
        
        # # Define the fully connected layer
        # self.fc1 = nn.Linear(input_size, input_size)
    
        # self.fc2 = nn.Linear(input_size, intermediate_size)

        self.fc3 = nn.Linear(intermediate_size, 1)
        self.sig = nn.Sigmoid()


        
    def forward(self, x, similarity_features):
        # Pass the input through the pretrained model 
        quality, quality_features = self.quality_model(x)

        # Process quality and similarity features
        similarity_features = self.fc_clip_feat(similarity_features)
        similarity_features = self.relu_clip_feat(similarity_features)
        similarity_features = self.drop_clip_feat(similarity_features)

        # Pass the similarity_features through fc final
        simi_features = self.fc_final(similarity_features)

        # Compute mean and std for quality features
        quality_mean, quality_std = calc_mean_std(quality_features)

        # Compute mean and std for similarity features
        similarity_mean, similarity_std = calc_mean_std(simi_features)

        # Normalize the quality features using AdaIN
        normalized_quality_features = adain(simi_features, quality_features)
        # print(normalized_quality_features.shape)

        # Pass the normalized quality features through the fully connected layers
        # x = self.fc1(x)
        # x = self.fc2(x)
        x = self.fc3(normalized_quality_features)
        return self.sig(x)
        # return x
    
def fetch_example(data):
    inputs = data['image']
    similarity_features = data['clip_image_features']
    batch_size = inputs.size()[0]
    labels = data['rating'].view(batch_size, -1) / 100.0
    
    inputs, similarity_features, labels = Variable(inputs.float().cuda()), Variable(similarity_features.float().cuda()), Variable(labels.float().cuda())
    return [inputs, similarity_features], labels

finetune_model(FusionModel, fetch_example, exp_name, num_epochs = num_epochs, num_splits = num_splits)

decay_rate is set to 1.0
Train Results - Split: 0, Epoch: 0, PLCC: 0.8249, SROCC: 0.8270, MSE: 0.0153
Validation Results - Split: 0, Epoch: 0, PLCC: 0.7811, SROCC: 0.7810, MSE: 0.0184, Best PLCC: 0.7811, Best SROCC: 0.7810, Best MSE: 0.0184
Train Results - Split: 0, Epoch: 1, PLCC: 0.9118, SROCC: 0.9002, MSE: 0.0099
Validation Results - Split: 0, Epoch: 1, PLCC: 0.8314, SROCC: 0.8146, MSE: 0.0163, Best PLCC: 0.8314, Best SROCC: 0.8146, Best MSE: 0.0163
Train Results - Split: 0, Epoch: 2, PLCC: 0.9454, SROCC: 0.9330, MSE: 0.0056
Validation Results - Split: 0, Epoch: 2, PLCC: 0.8478, SROCC: 0.8221, MSE: 0.0136, Best PLCC: 0.8478, Best SROCC: 0.8221, Best MSE: 0.0136
Train Results - Split: 0, Epoch: 3, PLCC: 0.9577, SROCC: 0.9517, MSE: 0.0054
Validation Results - Split: 0, Epoch: 3, PLCC: 0.8467, SROCC: 0.8229, MSE: 0.0149, Best PLCC: 0.8478, Best SROCC: 0.8229, Best MSE: 0.0136
Train Results - Split: 0, Epoch: 4, PLCC: 0.9625, SROCC: 0.9573, MSE: 0.0031
Validation Results - Split: 0, Epo

In [2]:
exp_name = "CLIP_features_diff_Meta-AI1k_(ViT-16)-Oct"

class FusionModel(nn.Module):
    def __init__(self):
        super(FusionModel, self).__init__()
        
        # Load the pretrained ResNet18 model
        self.quality_model = MetaIQA('model_IQA/TID2013_KADID10K_IQA_Meta_resnet18.pt')
        input_size = 1024
        intermediate_size = 512

        self.fc_clip_feat = nn.Linear(512, 512)
        self.fc_clip_feat.weight.data.normal_(0, 0.0)
        self.fc_clip_feat.bias.data.zero_()
        # self.bn_clip_feat = nn.BatchNorm1d(512)
        self.relu_clip_feat = nn.PReLU()
        self.drop_clip_feat = nn.Dropout(p=0.5)
        self.fc_final = nn.Linear(512,512)
        
        # # Define the fully connected layer
        self.fc1 = nn.Linear(input_size, input_size)
        self.fc2 = nn.Linear(input_size, intermediate_size)
        self.fc3 = nn.Linear(intermediate_size, 1)
        self.sigmoid = nn.Sigmoid()

        
    def forward(self, x, similarity_features):
        # Pass the input through the pretrained model 
        quality, quality_features = self.quality_model(x)
        # print (quality_features.shape)

        similarity_features = self.fc_clip_feat(similarity_features)
        # similarity_features = self.bn_clip_feat(similarity_features)
        similarity_features = self.relu_clip_feat(similarity_features)
        similarity_features = self.drop_clip_feat(similarity_features)

        final = self.fc_final(similarity_features)
        # Concatenate the final prediction output and the new input
        combined = torch.cat((quality_features, final), dim=1)

        x = self.fc1(combined)
        x = self.fc2(x)
        x = self.fc3(x)
        return self.sigmoid(x)
        # return x
    
def fetch_example(data):
    inputs = data['image']
    similarity_features = data['clip_image_features'] - data['clip_text_features']
    batch_size = inputs.size()[0]
    labels = data['rating'].view(batch_size, -1) / 5.0
    
    inputs, similarity_features, labels = Variable(inputs.float().cuda()), Variable(similarity_features.float().cuda()), Variable(labels.float().cuda())
    return [inputs, similarity_features], labels

finetune_model(FusionModel, fetch_example, exp_name, num_epochs = num_epochs, num_splits = num_splits)

decay_rate is set to 1.0
Train Results - Split: 0, Epoch: 0, PLCC: 0.8943, SROCC: 0.8596, MSE: 0.0062
Validation Results - Split: 0, Epoch: 0, PLCC: 0.8191, SROCC: 0.7528, MSE: 0.0093, Best PLCC: 0.8191, Best SROCC: 0.7528, Best MSE: 0.0093
Train Results - Split: 0, Epoch: 1, PLCC: 0.9358, SROCC: 0.9181, MSE: 0.0037
Validation Results - Split: 0, Epoch: 1, PLCC: 0.8312, SROCC: 0.7841, MSE: 0.0088, Best PLCC: 0.8312, Best SROCC: 0.7841, Best MSE: 0.0088
Train Results - Split: 0, Epoch: 2, PLCC: 0.9563, SROCC: 0.9523, MSE: 0.0029
Validation Results - Split: 0, Epoch: 2, PLCC: 0.8466, SROCC: 0.8139, MSE: 0.0088, Best PLCC: 0.8466, Best SROCC: 0.8139, Best MSE: 0.0088
Train Results - Split: 0, Epoch: 3, PLCC: 0.9589, SROCC: 0.9506, MSE: 0.0024
Validation Results - Split: 0, Epoch: 3, PLCC: 0.8300, SROCC: 0.8023, MSE: 0.0086, Best PLCC: 0.8466, Best SROCC: 0.8139, Best MSE: 0.0086
Train Results - Split: 0, Epoch: 4, PLCC: 0.9727, SROCC: 0.9727, MSE: 0.0017
Validation Results - Split: 0, Epo

: 

In [2]:
exp_name = "CLIP_features_2fc_Meta_AI1k-(ViT-16)-mean"

class FusionModel(nn.Module):
    def __init__(self):
        super(FusionModel, self).__init__()
        
        # Load the pretrained ResNet18 model
        self.quality_model = MetaIQA('model_IQA/TID2013_KADID10K_IQA_Meta_resnet18.pt')
        input_size = 1536

        intermediate_size = 512

        self.fc_clip_image = nn.Linear(512, 512)
        self.fc_clip_text = nn.Linear(512, 512)
        self.fc_clip_text.weight.data.normal_(0, 0.0)
        self.fc_clip_text.bias.data.zero_()
        
        self.relu_clip_feat = nn.PReLU()
        self.drop_clip_feat = nn.Dropout(p=0.5)

        self.fc_final = nn.Linear(1024, 1024)
        # self.sig = nn.Sigmoid()
        # self.attention = nn.MultiheadAttention(embed_dim=input_size, num_heads=1)

        # Define the fully connected layers
        self.fc1 = nn.Linear(input_size, input_size)
        self.fc2 = nn.Linear(input_size, intermediate_size)
        self.fc3 = nn.Linear(intermediate_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, clip_image, clip_text):
        # Pass the input through the pretrained model
        quality, quality_features = self.quality_model(x)

        image_out = self.fc_clip_image(clip_image)
        # # similarity_features = self.bn_clip_feat(similarity_features)
        image_out = self.relu_clip_feat(image_out)
        image_out = self.drop_clip_feat(image_out)

        text_out = self.fc_clip_text(clip_text)
        # # similarity_features = self.bn_clip_feat(similarity_features)
        text_out = self.relu_clip_feat(text_out)
        text_out = self.drop_clip_feat(text_out)

        combined = torch.cat((image_out, text_out), dim=1)
        fc_final = self.fc_final(combined)

        combined_1 = torch.cat((quality_features, fc_final), dim=1)


        
        # Pass the concatenated tensor through the fully connected layer
        x = self.fc1(combined_1)
        x = self.fc2(x)
        x = self.fc3(x)
        return self.sigmoid(x)
        # return x
    
def fetch_example(data):
    inputs = data['image']
    clip_image = data['clip_image_features']
    clip_text = data['clip_text_features']
    batch_size = inputs.size()[0]
    labels = data['rating'].view(batch_size, -1) / 5.0
    
    inputs, clip_image, clip_text, labels = Variable(inputs.float().cuda()), Variable(clip_image.float().cuda()),Variable(clip_text.float().cuda()), Variable(labels.float().cuda())
    return [inputs, clip_image, clip_text], labels

finetune_model(FusionModel, fetch_example, exp_name, num_epochs = num_epochs, num_splits = num_splits)


decay_rate is set to 1.0
Train Results - Split: 0, Epoch: 0, PLCC: 0.8990, SROCC: 0.8637, MSE: 0.0060
Validation Results - Split: 0, Epoch: 0, PLCC: 0.8292, SROCC: 0.7751, MSE: 0.0087, Best PLCC: 0.8292, Best SROCC: 0.7751, Best MSE: 0.0087
Train Results - Split: 0, Epoch: 1, PLCC: 0.9360, SROCC: 0.9171, MSE: 0.0038
Validation Results - Split: 0, Epoch: 1, PLCC: 0.8337, SROCC: 0.7894, MSE: 0.0094, Best PLCC: 0.8337, Best SROCC: 0.7894, Best MSE: 0.0087
Train Results - Split: 0, Epoch: 2, PLCC: 0.9632, SROCC: 0.9538, MSE: 0.0021
Validation Results - Split: 0, Epoch: 2, PLCC: 0.8336, SROCC: 0.7751, MSE: 0.0087, Best PLCC: 0.8337, Best SROCC: 0.7894, Best MSE: 0.0087
Train Results - Split: 0, Epoch: 3, PLCC: 0.9742, SROCC: 0.9681, MSE: 0.0018
Validation Results - Split: 0, Epoch: 3, PLCC: 0.8403, SROCC: 0.8004, MSE: 0.0083, Best PLCC: 0.8403, Best SROCC: 0.8004, Best MSE: 0.0083
Train Results - Split: 0, Epoch: 4, PLCC: 0.9743, SROCC: 0.9677, MSE: 0.0025
Validation Results - Split: 0, Epo

TypeError: __init__() missing 2 required positional arguments: 'similarity_features' and 'quality_features'

In [2]:
class CompactBilinearPooling(nn.Module):
    def __init__(self, input1_size, input2_size, output_size):
        super(CompactBilinearPooling, self).__init__()
        
        self.proj1 = nn.Linear(input1_size, output_size)
        self.proj2 = nn.Linear(input2_size, output_size)
        
    def forward(self, input1, input2):
        proj1 = self.proj1(input1)
        proj2 = self.proj2(input2)
        
        pooled = torch.einsum('bd,bd->bd', proj1, proj2)
        
        return pooled

exp_name = "HPS_Image_CNN_IQA_-CrossModalAttention_LIVE"

class FusionModel(nn.Module):
    def __init__(self):
        super(FusionModel, self).__init__()

        self.quality_model = MetaIQA('model_IQA/TID2013_KADID10K_IQA_Meta_resnet18.pt')
        input_size = 1024
        intermediate_size = 512

        self.fc_clip_feat = nn.Linear(512, 512)
        self.relu_clip_feat = nn.PReLU()
        self.drop_clip_feat = nn.Dropout(p=0.5)
        # self.fc_final = nn.Linear(768, 768)

        # Define the fully connected layer
        self.fc1 = nn.Linear(input_size, input_size)
        self.fc2 = nn.Linear(input_size, intermediate_size)

        self.fc3 = nn.Linear(intermediate_size, 1)
        self.sig = nn.Sigmoid()

    def forward(self, x, similarity_features):
        # Pass the input through the pretrained model
        quality, quality_features = self.quality_model(x)

        similarity_features = self.fc_clip_feat(similarity_features)
        similarity_features = self.relu_clip_feat(similarity_features)
        similarity_features = self.drop_clip_feat(similarity_features)
        # similarity_features = self.fc_final(similarity_features)

        # Apply cross-modal attention
        attended_quality, attended_similarity = self.cross_modal_attention(quality_features, similarity_features)

        # Concatenate the attended features
        combined = torch.cat((attended_quality, attended_similarity), dim=1)

        x = self.fc1(combined)
        x = self.fc2(x)
        x = self.fc3(x)
        return self.sig(x)

def fetch_example(data):
    inputs = data['image']
    similarity_features = data['clip_image_features']
    batch_size = inputs.size()[0]
    labels = data['rating'].view(batch_size, -1) / 100.0

    inputs, similarity_features, labels = Variable(inputs.float().cuda()), Variable(similarity_features.float().cuda()), Variable(labels.float().cuda())
    return [inputs, similarity_features], labels

finetune_model(FusionModel, fetch_example, exp_name, num_epochs=num_epochs, num_splits=num_splits)


decay_rate is set to 1.0


TypeError: empty(): argument 'size' must be tuple of ints, but found element of type Tensor at pos 2

: 

In [2]:
exp_name = "CLIP_ImAGE_MEta_Fusion_maE-LIVE"

class FusionModel(nn.Module):
    def __init__(self, ):
        super(FusionModel, self).__init__()
        
        # Load the pretrained ResNet18 model
        self.quality_model = MetaIQA('model_IQA/TID2013_KADID10K_IQA_Meta_resnet18.pt')
        input_size = 1024
   
        
        # Encoder layers for perceptual features
        self.encoder_perceptual = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
        )

        # Encoder layers for semantic features
        self.encoder_semantic = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
        )

        # Latent space
        self.latent_mean = nn.Linear(256, 128)
        self.latent_logvar = nn.Linear(256, 128)

        # Decoder layers for fused features
        self.decoder = nn.Sequential(
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
        )

        self.fc1 = nn.Linear(1024, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 1)
        self.sig = nn.Sigmoid()
        

        

    def reparameterize(self, mean, logvar):
        std = torch.exp(0.5 * logvar)
        epsilon = torch.randn_like(std)
        return mean + epsilon * std

    def forward(self, x, similarity_features):
        # Pass the input through the pretrained model 
        quality, quality_features = self.quality_model(x)        
        # Encode semantic features
        semantic_encoded = self.encoder_semantic(similarity_features)
        
        # Encode perceptual features
        perceptual_encoded = self.encoder_perceptual(quality_features)
        
        # Combine encoded features into the latent space
        combined_encoded = torch.cat([semantic_encoded, perceptual_encoded], dim=1)
        mean = self.latent_mean(combined_encoded)
        logvar = self.latent_logvar(combined_encoded)
        
        # Reparameterize to get latent variables
        z = self.reparameterize(mean, logvar)
        
        # Decode the latent variables to reconstruct features
        reconstructed = self.decoder(z)

        x = self.fc1(reconstructed)
        x = self.fc2(x)
        x = self.fc3(x)
        return self.sig(x)
        # return x
    
def fetch_example(data):
    inputs = data['image']
    similarity_features = data['clip_image_features']
    batch_size = inputs.size()[0]
    labels = data['rating'].view(batch_size, -1) / 100.0
    
    inputs, similarity_features, labels = Variable(inputs.float().cuda()), Variable(similarity_features.float().cuda()), Variable(labels.float().cuda())
    return [inputs, similarity_features], labels

finetune_model(FusionModel, fetch_example, exp_name, num_epochs = num_epochs, num_splits = num_splits)
        

decay_rate is set to 1.0
Train Results - Split: 0, Epoch: 0, PLCC: 0.0758, SROCC: 0.0871, MSE: 0.0409
Validation Results - Split: 0, Epoch: 0, PLCC: 0.0946, SROCC: 0.0748, MSE: 0.0437, Best PLCC: 0.0946, Best SROCC: 0.0748, Best MSE: 0.0437
Train Results - Split: 0, Epoch: 1, PLCC: 0.3525, SROCC: 0.3359, MSE: 0.0398
Validation Results - Split: 0, Epoch: 1, PLCC: 0.3731, SROCC: 0.3811, MSE: 0.0426, Best PLCC: 0.3731, Best SROCC: 0.3811, Best MSE: 0.0426
Train Results - Split: 0, Epoch: 2, PLCC: 0.7911, SROCC: 0.6564, MSE: 0.0185
Validation Results - Split: 0, Epoch: 2, PLCC: 0.7209, SROCC: 0.6161, MSE: 0.0240, Best PLCC: 0.7209, Best SROCC: 0.6161, Best MSE: 0.0240
Train Results - Split: 0, Epoch: 3, PLCC: 0.8464, SROCC: 0.8057, MSE: 0.0130
Validation Results - Split: 0, Epoch: 3, PLCC: 0.7238, SROCC: 0.7127, MSE: 0.0213, Best PLCC: 0.7238, Best SROCC: 0.7127, Best MSE: 0.0213
Train Results - Split: 0, Epoch: 4, PLCC: 0.8771, SROCC: 0.8051, MSE: 0.0128
Validation Results - Split: 0, Epo

KeyboardInterrupt: 

: 

In [4]:
exp_name = "HPS_Image_CNN_IQA-LIVE"

class FusionModel(nn.Module):
    def __init__(self):
        super(FusionModel, self).__init__()
        
        # Load the pretrained ResNet18 model
        # self.quality_model = MetaIQA('model_IQA/TID2013_KADID10K_IQA_Meta_resnet18.pt')
        self.quality_model = CNNIQAnet()
        input_size = 1280
        intermediate_size = 512

        self.fc_clip_feat = nn.Linear(768, 768)
     
        self.relu_clip_feat = nn.PReLU()
        self.drop_clip_feat = nn.Dropout(p=0.5)
        self.fc_final = nn.Linear(768, 768)
        
        # # Define the fully connected layer
        self.fc1 = nn.Linear(input_size, input_size)
    
        self.fc2 = nn.Linear(input_size, intermediate_size)

        self.fc3 = nn.Linear(intermediate_size, 1)
        self.sig = nn.Sigmoid()


        
    def forward(self, x, similarity_features):
        # Pass the input through the pretrained model 
        quality, quality_features = self.quality_model(x)

        similarity_features = self.fc_clip_feat(similarity_features)
        # similarity_features = self.bn_clip_feat(similarity_features)
        similarity_features = self.relu_clip_feat(similarity_features)
        similarity_features = self.drop_clip_feat(similarity_features)

        similarity_features = self.fc_final(similarity_features)

        # Concatenate the final prediction output and the new input
        combined = torch.cat((quality_features, similarity_features), dim=1)
        
        x = self.fc1(combined)
        x = self.fc2(x)
        x = self.fc3(x)
        return self.sig(x)
        # return x
    
def fetch_example(data):
    inputs = data['image']
    similarity_features = data['clip_image_features']
    batch_size = inputs.size()[0]
    labels = data['rating'].view(batch_size, -1) / 100.0
    
    inputs, similarity_features, labels = Variable(inputs.float().cuda()), Variable(similarity_features.float().cuda()), Variable(labels.float().cuda())
    return [inputs, similarity_features], labels

finetune_model(FusionModel, fetch_example, exp_name, num_epochs = num_epochs, num_splits = num_splits)

decay_rate is set to 1.0
Train Results - Split: 0, Epoch: 0, PLCC: 0.5691, SROCC: 0.5632, MSE: 0.0383
Validation Results - Split: 0, Epoch: 0, PLCC: 0.4636, SROCC: 0.4946, MSE: 0.0417, Best PLCC: 0.4636, Best SROCC: 0.4946, Best MSE: 0.0417
Train Results - Split: 0, Epoch: 1, PLCC: 0.6794, SROCC: 0.6787, MSE: 0.0334
Validation Results - Split: 0, Epoch: 1, PLCC: 0.6259, SROCC: 0.6511, MSE: 0.0365, Best PLCC: 0.6259, Best SROCC: 0.6511, Best MSE: 0.0365
Train Results - Split: 0, Epoch: 2, PLCC: 0.7432, SROCC: 0.7382, MSE: 0.0214
Validation Results - Split: 0, Epoch: 2, PLCC: 0.7112, SROCC: 0.7246, MSE: 0.0240, Best PLCC: 0.7112, Best SROCC: 0.7246, Best MSE: 0.0240
Train Results - Split: 0, Epoch: 3, PLCC: 0.8005, SROCC: 0.7862, MSE: 0.0160
Validation Results - Split: 0, Epoch: 3, PLCC: 0.7713, SROCC: 0.7747, MSE: 0.0188, Best PLCC: 0.7713, Best SROCC: 0.7747, Best MSE: 0.0188
Train Results - Split: 0, Epoch: 4, PLCC: 0.8354, SROCC: 0.8179, MSE: 0.0157
Validation Results - Split: 0, Epo

: 

# EXP 2

In [2]:
class LowRankBilinearPooling(nn.Module):
    def __init__(self, quality_input_size, similarity_input_size, output_size, dropout_rate=0.0):
        super(LowRankBilinearPooling, self).__init__()

        self.quality_input_size = quality_input_size
        self.similarity_input_size = similarity_input_size
        self.output_size = output_size

        # Create linear layers for quality and similarity features
        # self.quality_ff = nn.Linear(self.quality_input_size, self.output_size)
        self.similarity_ff = nn.Linear(self.similarity_input_size, self.output_size)
        

        # Create activation layer
        self.activation = nn.PReLU()

        # Create dropout layer
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, quality_features, similarity_features):
        # Apply nonlinearities and dropout on quality features
        quality_features = self.activation(quality_features)
        quality_features = self.dropout(quality_features)

        # Apply nonlinearities and dropout on similarity features
        similarity_features = self.activation(similarity_features)
        similarity_features = self.dropout(similarity_features)

        # Pass quality features through the quality linear layer
        # latent_quality = self.quality_ff(quality_features)

        # Pass similarity features through the similarity linear layer
        latent_similarity = self.similarity_ff(similarity_features)

        # Element-wise multiplication
        outputs = quality_features * latent_similarity

        return outputs


class FusionModel(nn.Module):
    def __init__(self):
        super(FusionModel, self).__init__()

        # Load the pretrained ResNet18 model
        self.quality_model = MetaIQA('model_IQA/TID2013_KADID10K_IQA_Meta_resnet18.pt')
        quality_input_size = 512
        clip_input_size = 512

        # Initialize the LowRankBilinearPooling model
        self.fusion_model = LowRankBilinearPooling(quality_input_size, clip_input_size, output_size=512, dropout_rate=0.5)

        # Define the fully connected layers for fusion
        self.fc1 = nn.Linear(1024, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 1)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x, similarity_features):
        # Pass the input through the pretrained model
        quality, quality_features = self.quality_model(x)

        # Pass the quality and similarity features through the fusion model (LowRankBilinearPooling)
        fused_features = self.fusion_model(quality_features, similarity_features)

        # Concatenate the fused features with the quality features
        fused_features = torch.cat((quality_features, fused_features), dim=1)

        # Pass the fused features through the fully connected layers
        x = self.fc1(fused_features)
        x = self.fc2(x)

        return self.sigmoid(x)

def fetch_example(data):
    inputs = data['image']
    similarity_features = data['clip_image_features']
    batch_size = inputs.size()[0]
    labels = data['rating'].view(batch_size, -1) / 100.0

    inputs, similarity_features, labels = Variable(inputs.float().cuda()), Variable(similarity_features.float().cuda()), Variable(labels.float().cuda())
    return [inputs, similarity_features], labels

exp_name = "CLIP_features_Image_IQA_LIVE_lowRank"

finetune_model(FusionModel, fetch_example, exp_name, num_epochs=num_epochs, num_splits=num_splits)


decay_rate is set to 1.0
Train Results - Split: 0, Epoch: 0, PLCC: 0.8557, SROCC: 0.8346, MSE: 0.0262
Validation Results - Split: 0, Epoch: 0, PLCC: 0.7981, SROCC: 0.7808, MSE: 0.0301, Best PLCC: 0.7981, Best SROCC: 0.7808, Best MSE: 0.0301
Train Results - Split: 0, Epoch: 1, PLCC: 0.9260, SROCC: 0.9084, MSE: 0.0226
Validation Results - Split: 0, Epoch: 1, PLCC: 0.7997, SROCC: 0.7796, MSE: 0.0319, Best PLCC: 0.7997, Best SROCC: 0.7808, Best MSE: 0.0301
Train Results - Split: 0, Epoch: 2, PLCC: 0.9377, SROCC: 0.9196, MSE: 0.0052
Validation Results - Split: 0, Epoch: 2, PLCC: 0.8012, SROCC: 0.7667, MSE: 0.0152, Best PLCC: 0.8012, Best SROCC: 0.7808, Best MSE: 0.0152
Train Results - Split: 0, Epoch: 3, PLCC: 0.9398, SROCC: 0.9147, MSE: 0.0072
Validation Results - Split: 0, Epoch: 3, PLCC: 0.7953, SROCC: 0.7334, MSE: 0.0204, Best PLCC: 0.8012, Best SROCC: 0.7808, Best MSE: 0.0152
Train Results - Split: 0, Epoch: 4, PLCC: 0.9601, SROCC: 0.9492, MSE: 0.0046
Validation Results - Split: 0, Epo

KeyboardInterrupt: 

: 

In [ ]:
exp_name = "CLIP_Image_IQA_(ViT-16)-LIVE"

class FusionModel(nn.Module):
    def __init__(self):
        super(FusionModel, self).__init__()
        
        # Load the pretrained ResNet18 model
        self.quality_model = MetaIQA('model_IQA/TID2013_KADID10K_IQA_Meta_resnet18.pt')
        input_size = 1024
        intermediate_size = 512

        self.fc_clip_feat = nn.Linear(512, 512)
     
        self.relu_clip_feat = nn.PReLU()
        self.drop_clip_feat = nn.Dropout(p=0.5)
        self.fc_final = nn.Linear(512, 512)
        
        # # Define the fully connected layer
        self.fc1 = nn.Linear(input_size, input_size)
    
        self.fc2 = nn.Linear(input_size, intermediate_size)

        self.fc3 = nn.Linear(intermediate_size, 1)
        self.sig = nn.Sigmoid()


        
    def forward(self, x, similarity_features):
        # Pass the input through the pretrained model 
        quality, quality_features = self.quality_model(x)

        similarity_features = self.fc_clip_feat(similarity_features)
        # similarity_features = self.bn_clip_feat(similarity_features)
        similarity_features = self.relu_clip_feat(similarity_features)
        similarity_features = self.drop_clip_feat(similarity_features)

        similarity_features = self.fc_final(similarity_features)

        # Concatenate the final prediction output and the new input
        combined = torch.cat((quality_features, similarity_features), dim=1)
        
        x = self.fc1(combined)
        x = self.fc2(x)
        x = self.fc3(x)
        return self.sig(x)
        # return x
    
def fetch_example(data):
    inputs = data['image']
    similarity_features = data['clip_image_features']
    batch_size = inputs.size()[0]
    labels = data['rating'].view(batch_size, -1) / 100.0
    
    inputs, similarity_features, labels = Variable(inputs.float().cuda()), Variable(similarity_features.float().cuda()), Variable(labels.float().cuda())
    return [inputs, similarity_features], labels

finetune_model(FusionModel, fetch_example, exp_name, num_epochs = num_epochs, num_splits = num_splits)

In [2]:
class FactorizedBilinearPooling(nn.Module):
    def __init__(self, quality_input_size, similarity_input_size, output_size, dropout_rate=0.0):
        super(FactorizedBilinearPooling, self).__init__()

        self.quality_input_size = quality_input_size
        self.similarity_input_size = similarity_input_size
        self.output_size = output_size


        # Factorized bilinear pooling matrices
        self.W1 = nn.Linear(self.quality_input_size * self.similarity_input_size, self.output_size)
        self.W2 = nn.Linear(self.quality_input_size * self.similarity_input_size, self.output_size)

        # Create activation layer
        self.activation = nn.PReLU()

        # Create dropout layer
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, quality_features, similarity_features):
        # Apply nonlinearities and dropout on quality features
        # quality_features = self.activation(quality_features)
        # quality_features = self.dropout(quality_features)

        # Apply nonlinearities and dropout on similarity features
        similarity_features = self.activation(similarity_features)
        similarity_features = self.dropout(similarity_features)

        # Factorized bilinear pooling: outer product and factorization
        quality_outer = quality_features.view(quality_features.size(0), -1)
        similarity_outer = similarity_features.view(similarity_features.size(0), -1)
        bilinear = torch.bmm(quality_outer.unsqueeze(2), similarity_outer.unsqueeze(1))
        bilinear = bilinear.view(bilinear.size(0), -1)
        pooled_features = self.activation(self.W1(bilinear)) + self.activation(self.W2(bilinear))

        return pooled_features


class FusionModel(nn.Module):
    def __init__(self):
        super(FusionModel, self).__init__()

        # Load the pretrained ResNet18 model
        self.quality_model = MetaIQA('model_IQA/TID2013_KADID10K_IQA_Meta_resnet18.pt')
        quality_input_size = 512
        clip_input_size = 512

        # Initialize the FactorizedBilinearPooling model
        self.fusion_model = FactorizedBilinearPooling(quality_input_size, clip_input_size, output_size=1024, dropout_rate=0.5)

        # Define the fully connected layers for fusion
        self.fc1 = nn.Linear(1536, 1536)
        self.fc2 = nn.Linear(1536, 1024)
        self.fc3 = nn.Linear(1024, 1)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x, similarity_features):
        # Pass the input through the pretrained model
        quality, quality_features = self.quality_model(x)

        # Pass the quality and similarity features through the fusion model (FactorizedBilinearPooling)
        fused_features = self.fusion_model(quality_features, similarity_features)

        # Concatenate the fused features with the quality features
        fused_features = torch.cat((quality_features, fused_features), dim=1)

        # Pass the fused features through the fully connected layers
        x = self.fc1(fused_features)
        x = self.fc2(x)
        x = self.fc3(x)

        return self.sigmoid(x)
    
def fetch_example(data):
    inputs = data['image']
    similarity_features = data['clip_image_features']
    batch_size = inputs.size()[0]
    labels = data['rating'].view(batch_size, -1) / 100.0

    inputs, similarity_features, labels = Variable(inputs.float().cuda()), Variable(similarity_features.float().cuda()), Variable(labels.float().cuda())
    return [inputs, similarity_features], labels

exp_name = "CLIP_features_Image_IQA_LIVE_factorizedPooling"

finetune_model(FusionModel, fetch_example, exp_name, num_epochs=num_epochs, num_splits=num_splits)


decay_rate is set to 1.0


RuntimeError: CUDA out of memory. Tried to allocate 1024.00 MiB (GPU 0; 11.76 GiB total capacity; 9.29 GiB already allocated; 388.12 MiB free; 9.30 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

: 

# EXP 3

In [3]:
exp_name = "HPS_features_image_IQA_LIVE-Gated"

class FusionModel(nn.Module):
    def __init__(self):
        super(FusionModel, self).__init__()
        
        # Load the pretrained ResNet18 model
        self.quality_model = MetaIQA('model_IQA/TID2013_KADID10K_IQA_Meta_resnet18.pt')
        input_size = 512

        intermediate_size = 512

        self.fc_clip_feat = nn.Linear(768, 512)

        self.fc_quality_feat = nn.Linear(512, 512)
        
        
        #Gating mechanism
        self.gate_quality = nn.Linear(512, 512)
        self.gate_clip_image = nn.Linear(512, 512)



        # Define the fully connected layers
        self.fc1 = nn.Linear(input_size, input_size)
        self.fc2 = nn.Linear(input_size, 1)
        # self.fc3 = nn.Linear(intermediate_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, similarity_features):
        # Pass the input through the pretrained model 
        quality, quality_features = self.quality_model(x)

        similarity_features = self.fc_clip_feat(similarity_features)
        quality_features = self.fc_quality_feat(quality_features)
        # similarity_features = self.bn_clip_feat(similarity_features)

        #Gated Fusion
        gated_quality = self.sigmoid(self.gate_quality(quality_features))
        gated_similarity = self.sigmoid(self.gate_clip_image(similarity_features))
        # Concatenate the final prediction output and the new input
        # combined_1 = torch.cat((quality_features, similarity_features), dim=1)
        # fc_final = self.fc_final(similarity_features)

        #Fused features
        fused_features = gated_quality * quality_features + gated_similarity * similarity_features

        # combined_1 = torch.cat((quality_features, fc_final), dim=1)

        
        
        # # # # Pass the concatenated tensor through the fully connected layer
        x = self.fc1(fused_features)
        # # # # x = self.fc_final(combined2)
       
        x = self.fc2(x)
       
        # x = self.fc3(x)
        return self.sigmoid(x)
        # return x
    
def fetch_example(data):
    inputs = data['image']
    similarity_features = data['clip_image_features'] #- data['clip_text_features']
    batch_size = inputs.size()[0]
    labels = data['rating'].view(batch_size, -1) / 100.0
    
    inputs, similarity_features, labels = Variable(inputs.float().cuda()), Variable(similarity_features.float().cuda()), Variable(labels.float().cuda())
    return [inputs, similarity_features], labels

finetune_model(FusionModel, fetch_example, exp_name, num_epochs = num_epochs, num_splits = num_splits)

decay_rate is set to 1.0
Train Results - Split: 0, Epoch: 0, PLCC: 0.8540, SROCC: 0.8285, MSE: 0.0114
Validation Results - Split: 0, Epoch: 0, PLCC: 0.7849, SROCC: 0.7695, MSE: 0.0162, Best PLCC: 0.7849, Best SROCC: 0.7695, Best MSE: 0.0162
Train Results - Split: 0, Epoch: 1, PLCC: 0.9274, SROCC: 0.9095, MSE: 0.0057
Validation Results - Split: 0, Epoch: 1, PLCC: 0.7932, SROCC: 0.7556, MSE: 0.0159, Best PLCC: 0.7932, Best SROCC: 0.7695, Best MSE: 0.0159
Train Results - Split: 0, Epoch: 2, PLCC: 0.9576, SROCC: 0.9425, MSE: 0.0039
Validation Results - Split: 0, Epoch: 2, PLCC: 0.8191, SROCC: 0.7872, MSE: 0.0149, Best PLCC: 0.8191, Best SROCC: 0.7872, Best MSE: 0.0149
Train Results - Split: 0, Epoch: 3, PLCC: 0.9653, SROCC: 0.9581, MSE: 0.0034
Validation Results - Split: 0, Epoch: 3, PLCC: 0.8156, SROCC: 0.7767, MSE: 0.0155, Best PLCC: 0.8191, Best SROCC: 0.7872, Best MSE: 0.0149
Train Results - Split: 0, Epoch: 4, PLCC: 0.9616, SROCC: 0.9531, MSE: 0.0043
Validation Results - Split: 0, Epo

: 

In [2]:
exp_name = "CLIP_features2fc_AGIQA-(ViT-16)-atarashi"

class FusionModel(nn.Module):
    def __init__(self):
        super(FusionModel, self).__init__()
        
        # Load the pretrained ResNet18 model
        self.quality_model = MetaIQA('model_IQA/TID2013_KADID10K_IQA_Meta_resnet18.pt')
        input_size = 1536

        intermediate_size = 512

        self.fc_clip_image = nn.Linear(512, 512)
        self.fc_clip_image.weight.data.normal_(0, 0.0)
        self.fc_clip_image.bias.data.zero_()
        # self.bn_clip_feat = nn.BatchNorm1d(512)
        self.fc_clip_text = nn.Linear(512, 512)
        self.fc_clip_text.weight.data.normal_(0, 0.0)
        self.fc_clip_text.bias.data.zero_()
        
        self.relu_clip_feat = nn.PReLU()
        self.drop_clip_feat = nn.Dropout(p=0.5)

        self.fc_final = nn.Linear(1024, 1024)
        # self.sig = nn.Sigmoid()
        # self.attention = nn.MultiheadAttention(embed_dim=input_size, num_heads=1)

        # Define the fully connected layers
        self.fc1 = nn.Linear(input_size, input_size)
        self.fc2 = nn.Linear(input_size, intermediate_size)
        self.fc3 = nn.Linear(intermediate_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, clip_image, clip_text):
        # Pass the input through the pretrained model
        quality, quality_features = self.quality_model(x)

        image_out = self.fc_clip_image(clip_image)
        # # similarity_features = self.bn_clip_feat(similarity_features)
        image_out = self.relu_clip_feat(image_out)
        image_out = self.drop_clip_feat(image_out)

        text_out = self.fc_clip_text(clip_text)
        # # similarity_features = self.bn_clip_feat(similarity_features)
        text_out = self.relu_clip_feat(text_out)
        text_out = self.drop_clip_feat(text_out)

        combined = torch.cat((image_out, text_out), dim=1)
        fc_final = self.fc_final(combined)

        combined_1 = torch.cat((quality_features, fc_final), dim=1)


        
        # Pass the concatenated tensor through the fully connected layer
        x = self.fc1(combined_1)
        x = self.fc2(x)
        x = self.fc3(x)
        return self.sigmoid(x)
        # return x
    
def fetch_example(data):
    inputs = data['image']
    clip_image = data['clip_image_features']
    clip_text = data['clip_text_features']
    batch_size = inputs.size()[0]
    labels = data['rating'].view(batch_size, -1) / 5.0
    
    inputs, clip_image, clip_text, labels = Variable(inputs.float().cuda()), Variable(clip_image.float().cuda()),Variable(clip_text.float().cuda()), Variable(labels.float().cuda())
    return [inputs, clip_image, clip_text], labels

finetune_model(FusionModel, fetch_example, exp_name, num_epochs = num_epochs, num_splits = num_splits)


decay_rate is set to 1.0
Train Results - Split: 0, Epoch: 0, PLCC: 0.9357, SROCC: 0.8964, MSE: 0.0050
Validation Results - Split: 0, Epoch: 0, PLCC: 0.9222, SROCC: 0.8892, MSE: 0.0062, Best PLCC: 0.9222, Best SROCC: 0.8892, Best MSE: 0.0062
Train Results - Split: 0, Epoch: 1, PLCC: 0.9532, SROCC: 0.9276, MSE: 0.0037
Validation Results - Split: 0, Epoch: 1, PLCC: 0.9210, SROCC: 0.8887, MSE: 0.0062, Best PLCC: 0.9222, Best SROCC: 0.8892, Best MSE: 0.0062
Train Results - Split: 0, Epoch: 2, PLCC: 0.9644, SROCC: 0.9434, MSE: 0.0029
Validation Results - Split: 0, Epoch: 2, PLCC: 0.9146, SROCC: 0.8765, MSE: 0.0072, Best PLCC: 0.9222, Best SROCC: 0.8892, Best MSE: 0.0062
Train Results - Split: 0, Epoch: 3, PLCC: 0.9662, SROCC: 0.9459, MSE: 0.0028
Validation Results - Split: 0, Epoch: 3, PLCC: 0.9021, SROCC: 0.8516, MSE: 0.0083, Best PLCC: 0.9222, Best SROCC: 0.8892, Best MSE: 0.0062
Train Results - Split: 0, Epoch: 4, PLCC: 0.9748, SROCC: 0.9649, MSE: 0.0020
Validation Results - Split: 0, Epo

: 

In [2]:
exp_name = "CLIP_features_AIGC_features-(ViT-14)-cross-attention-FC11"

class CrossAttentionModel(nn.Module):
    def __init__(self, input_size):
        super(CrossAttentionModel, self).__init__()

        self.attention = nn.MultiheadAttention(embed_dim=input_size, num_heads=1)

    def forward(self, feature1, feature2):
        # Compute cross-attention
        attended_feature1, _ = self.attention(feature1.permute(1, 0, 2), feature2.permute(1, 0, 2))
        attended_feature1 = attended_feature1.permute(1, 0, 2)  # (batch_size, seq_len, input_size)

        return attended_feature1


class FusionModel(nn.Module):
    def __init__(self):
        super(FusionModel, self).__init__()

        # Load the pretrained ResNet18 model
        self.quality_model = MetaIQA('model_IQA/TID2013_KADID10K_IQA_Meta_resnet18.pt')

        input_size = 512 + 1536  # quality_features (512) + combined clip features (1536)
        intermediate_size = 512

        self.cross_attention = CrossAttentionModel(input_size)
        self.fc = nn.Linear(input_size, intermediate_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, clip_image, clip_text):
        quality, quality_features = self.quality_model(x)

        combined_clip = torch.cat((clip_image, clip_text), dim=1)  # (batch_size, 1536)

        # Compute attended features using attention mechanism
        attended_features = self.cross_attention(quality_features.unsqueeze(1), combined_clip.unsqueeze(1))

        # Reshape attended features
        attended_features = attended_features.squeeze(1)  # (batch_size, seq_len, input_size)

        # Pass attended features through the fully connected layer
        x = self.fc(attended_features)
        x = self.sigmoid(x)

        return x


def fetch_example(data):
    inputs = data['image']
    clip_image = data['clip_image_features']
    clip_text = data['clip_text_features']
    batch_size = inputs.size()[0]
    labels = data['rating'].view(batch_size, -1) / 5.0
    
    inputs, clip_image, clip_text, labels = (
        inputs.float().cuda(),
        clip_image.float().cuda(),
        clip_text.float().cuda(),
        labels.float().cuda()
    )
    return [inputs, clip_image, clip_text], labels

finetune_model(FusionModel, fetch_example, exp_name, num_epochs=num_epochs, num_splits=num_splits)


decay_rate is set to 1.0


TypeError: forward() missing 1 required positional argument: 'value'

: 

In [2]:
exp_name = "no_similarity_features-AGIQA-3k"

class FusionModel(nn.Module):
    def __init__(self):
        super(FusionModel, self).__init__()
        
        # Load the pretrained ResNet18 model
        self.quality_model = MetaIQA('model_IQA/TID2013_KADID10K_IQA_Meta_resnet18.pt')
        input_size = 512
        intermediate_size = 512

        self.sig = nn.Sigmoid()

        # for m in self.modules():
        #     if isinstance(m, nn.Conv2d):
        #         # Weight initialization reference: https://arxiv.org/abs/1502.01852
        #         n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
        #         m.weight.data.normal_(0, math.sqrt(2. / n))
        #     elif isinstance(m, nn.BatchNorm2d):
        #         m.weight.data.fill_(1)
        #         m.bias.data.zero_()
        
    def forward(self, x, similarity_features):
        # Pass the input through the pretrained model 
        quality, quality_features = self.quality_model(x)
        # print (quality_features.shape)

        # similarity_features = self.fc_clip_feat(similarity_features)
        # # similarity_features = self.bn_clip_feat(similarity_features)
        # similarity_features = self.relu_clip_feat(similarity_features)
        # similarity_features = self.drop_clip_feat(similarity_features)
        # Concatenate the final prediction output and the new input
        # x = torch.cat((quality_features, similarity_features), dim=1)
        # x = quality_features
        # print(x.shape)
        
 
        return self.sig(quality)
        # return x
    
def fetch_example(data):
    inputs = data['image']
    similarity_features = data['clip_image_features']- data['clip_text_features']
    batch_size = inputs.size()[0]
    labels = data['rating'].view(batch_size, -1) / 5.0
    
    inputs, similarity_features, labels = Variable(inputs.float().cuda()), Variable(similarity_features.float().cuda()), Variable(labels.float().cuda())
    return [inputs, similarity_features], labels

finetune_model(FusionModel, fetch_example, exp_name, num_epochs = num_epochs, num_splits = num_splits)

decay_rate is set to 1.0
Train Results - Split: 0, Epoch: 0, PLCC: 0.9017, SROCC: 0.8591, MSE: 0.0110
Validation Results - Split: 0, Epoch: 0, PLCC: 0.8681, SROCC: 0.8175, MSE: 0.0128, Best PLCC: 0.8681, Best SROCC: 0.8175, Best MSE: 0.0128
Train Results - Split: 0, Epoch: 1, PLCC: 0.9333, SROCC: 0.9065, MSE: 0.0062
Validation Results - Split: 0, Epoch: 1, PLCC: 0.8790, SROCC: 0.8336, MSE: 0.0097, Best PLCC: 0.8790, Best SROCC: 0.8336, Best MSE: 0.0097
Train Results - Split: 0, Epoch: 2, PLCC: 0.9501, SROCC: 0.9379, MSE: 0.0042
Validation Results - Split: 0, Epoch: 2, PLCC: 0.8845, SROCC: 0.8378, MSE: 0.0097, Best PLCC: 0.8845, Best SROCC: 0.8378, Best MSE: 0.0097
Train Results - Split: 0, Epoch: 3, PLCC: 0.9638, SROCC: 0.9492, MSE: 0.0039
Validation Results - Split: 0, Epoch: 3, PLCC: 0.8912, SROCC: 0.8374, MSE: 0.0102, Best PLCC: 0.8912, Best SROCC: 0.8378, Best MSE: 0.0097
Train Results - Split: 0, Epoch: 4, PLCC: 0.9674, SROCC: 0.9571, MSE: 0.0040
Validation Results - Split: 0, Epo

: 

In [2]:
exp_name = "no_similarity_features-AVA"

class FusionModel(nn.Module):
    def __init__(self):
        super(FusionModel, self).__init__()
        
        # Load the pretrained ResNet18 model
        self.quality_model = MetaIQA('model_IQA/TID2013_KADID10K_IQA_Meta_resnet18.pt')
        input_size = 512
        intermediate_size = 512

        # self.fc_clip_feat = nn.Linear(512, 512)
        # self.fc_clip_feat.weight.data.normal_(0, 0.0)
        # self.fc_clip_feat.bias.data.zero_()
        # # self.bn_clip_feat = nn.BatchNorm1d(512)
        # self.relu_clip_feat = nn.PReLU()
        # self.drop_clip_feat = nn.Dropout(p=0.5)
        
        # Define the fully connected layer
        # self.fc1 = nn.Linear(input_size, input_size)
        # self.bn1 = nn.BatchNorm1d(input_size)
        # self.relu1 = nn.PReLU()
        # self.drop1 = nn.Dropout(p=0.5)
        # self.fc2 = nn.Linear(input_size, intermediate_size)
        # self.bn2 = nn.BatchNorm1d(intermediate_size)
        # self.relu2 = nn.PReLU()
        # self.drop2 = nn.Dropout(p=0.5)
        # self.fc3 = nn.Linear(intermediate_size, 1)
        self.sig = nn.Sigmoid()

        # for m in self.modules():
        #     if isinstance(m, nn.Conv2d):
        #         # Weight initialization reference: https://arxiv.org/abs/1502.01852
        #         n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
        #         m.weight.data.normal_(0, math.sqrt(2. / n))
        #     elif isinstance(m, nn.BatchNorm2d):
        #         m.weight.data.fill_(1)
        #         m.bias.data.zero_()
        
    def forward(self, x, vila_features):
        # Pass the input through the pretrained model 
        quality, quality_features = self.quality_model(x)
        # print (quality_features.shape)

        # similarity_features = self.fc_clip_feat(similarity_features)
        # # similarity_features = self.bn_clip_feat(similarity_features)
        # similarity_features = self.relu_clip_feat(similarity_features)
        # similarity_features = self.drop_clip_feat(similarity_features)
        # Concatenate the final prediction output and the new input
        # x = torch.cat((quality_features, similarity_features), dim=1)
        # x = quality_features
        # print(x.shape)
        
        # Pass the concatenated tensor through the fully connected layer
        # x = self.fc1(x)
        # x = self.bn1(x)
        # x = self.relu1(x)
        # x = self.drop1(x)
        # x = self.fc2(x)
        # x = self.bn2(x)
        # x = self.relu2(x)
        # x = self.drop2(x)
        # x = self.fc3(x)
        return self.sig(quality)
        # return x
    
def fetch_example(data):
    inputs = data['image']
    # similarity_features = data['clip_image_features']- data['clip_text_features'
    vila_features = data['ava_features']
    batch_size = inputs.size()[0]
    labels = data['rating'].view(batch_size, -1) / 10.0
    
    inputs, vila_features, labels = Variable(inputs.float().cuda()), Variable(vila_features.float().cuda()), Variable(labels.float().cuda())
    return [inputs, vila_features], labels

finetune_model(FusionModel, fetch_example, exp_name, num_epochs = num_epochs, num_splits = num_splits)

decay_rate is set to 1.0
Train Results - Split: 0, Epoch: 0, PLCC: 0.5512, SROCC: 0.5304, MSE: 0.0039
Validation Results - Split: 0, Epoch: 0, PLCC: 0.5389, SROCC: 0.5262, MSE: 0.0040, Best PLCC: 0.5389, Best SROCC: 0.5262, Best MSE: 0.0040
Train Results - Split: 0, Epoch: 1, PLCC: 0.5729, SROCC: 0.5538, MSE: 0.0038
Validation Results - Split: 0, Epoch: 1, PLCC: 0.5612, SROCC: 0.5492, MSE: 0.0039, Best PLCC: 0.5612, Best SROCC: 0.5492, Best MSE: 0.0039
Train Results - Split: 0, Epoch: 2, PLCC: 0.5861, SROCC: 0.5670, MSE: 0.0037
Validation Results - Split: 0, Epoch: 2, PLCC: 0.5661, SROCC: 0.5560, MSE: 0.0039, Best PLCC: 0.5661, Best SROCC: 0.5560, Best MSE: 0.0039


KeyboardInterrupt: 

In [2]:
exp_name = "CLIP_features-diff-+IQA-(ResNet-101)-LIVE-Recently"

class FusionModel(nn.Module):
    def __init__(self):
        super(FusionModel, self).__init__()
        
        # Load the pretrained ResNet18 model
        self.quality_model = MetaIQA('model_IQA/TID2013_KADID10K_IQA_Meta_resnet18.pt')
        input_size = 1024
        intermediate_size = 512

        self.fc_clip_feat = nn.Linear(512, 512)
        self.fc_clip_feat.weight.data.normal_(0, 0.0)
        self.fc_clip_feat.bias.data.zero_()
        # self.bn_clip_feat = nn.BatchNorm1d(512)
        self.relu_clip_feat = nn.PReLU()
        self.drop_clip_feat = nn.Dropout(p=0.5)
        self.fc_final = nn.Linear(512, 512)
        
        # # Define the fully connected layer
        self.fc1 = nn.Linear(input_size, input_size)
        # # self.bn1 = nn.BatchNorm1d(input_size)
        # # self.relu1 = nn.PReLU()
        # # self.drop1 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(input_size, intermediate_size)
        # # self.bn2 = nn.BatchNorm1d(intermediate_size)
        # # self.relu2 = nn.PReLU()
        # # self.drop2 = nn.Dropout(p=0.5)
        self.fc3 = nn.Linear(intermediate_size, 1)
        self.sig = nn.Sigmoid()

        # for m in self.modules():
        #     if isinstance(m, nn.Conv2d):
        #         # Weight initialization reference: https://arxiv.org/abs/1502.01852
        #         n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
        #         m.weight.data.normal_(0, math.sqrt(2. / n))
        #     elif isinstance(m, nn.BatchNorm2d):
        #         m.weight.data.fill_(1)
        #         m.bias.data.zero_()
        
    def forward(self, x, similarity_features):
        # Pass the input through the pretrained model 
        quality, quality_features = self.quality_model(x)
        # print (quality_features.shape)

        similarity_features = self.fc_clip_feat(similarity_features)
        # similarity_features = self.bn_clip_feat(similarity_features)
        similarity_features = self.relu_clip_feat(similarity_features)
        similarity_features = self.drop_clip_feat(similarity_features)

        similarity_features = self.fc_final(similarity_features)

        # comb = torch.cat((quality, similarity), dim=1)
        # Concatenate the final prediction output and the new input
        combined = torch.cat((quality_features, similarity_features), dim=1)
        # x = quality_features
        # print(x.shape)
        # final_combined = torch.cat((combined, comb), dim=1)
        # Pass the concatenated tensor through the fully connected layer
        x = self.fc1(combined)
        x = self.fc2(x)
        x = self.fc3(x)
        return self.sig(x)
        # return x
    
def fetch_example(data):
    inputs = data['image']
    similarity_features = data['clip_image_features'] - data['clip_text_features']
    batch_size = inputs.size()[0]
    labels = data['rating'].view(batch_size, -1) / 100.0
    
    inputs, similarity_features, labels = Variable(inputs.float().cuda()), Variable(similarity_features.float().cuda()), Variable(labels.float().cuda())
    return [inputs, similarity_features], labels

finetune_model(FusionModel, fetch_example, exp_name, num_epochs = num_epochs, num_splits = num_splits)

decay_rate is set to 1.0
Train Results - Split: 0, Epoch: 0, PLCC: 0.8547, SROCC: 0.8384, MSE: 0.0124
Validation Results - Split: 0, Epoch: 0, PLCC: 0.8108, SROCC: 0.7988, MSE: 0.0159, Best PLCC: 0.8108, Best SROCC: 0.7988, Best MSE: 0.0159
Train Results - Split: 0, Epoch: 1, PLCC: 0.9244, SROCC: 0.9060, MSE: 0.0066
Validation Results - Split: 0, Epoch: 1, PLCC: 0.7958, SROCC: 0.7665, MSE: 0.0160, Best PLCC: 0.8108, Best SROCC: 0.7988, Best MSE: 0.0159
Train Results - Split: 0, Epoch: 2, PLCC: 0.9404, SROCC: 0.9207, MSE: 0.0049
Validation Results - Split: 0, Epoch: 2, PLCC: 0.8126, SROCC: 0.7984, MSE: 0.0148, Best PLCC: 0.8126, Best SROCC: 0.7988, Best MSE: 0.0148
Train Results - Split: 0, Epoch: 3, PLCC: 0.9464, SROCC: 0.9302, MSE: 0.0068
Validation Results - Split: 0, Epoch: 3, PLCC: 0.8136, SROCC: 0.7985, MSE: 0.0179, Best PLCC: 0.8136, Best SROCC: 0.7988, Best MSE: 0.0148
Train Results - Split: 0, Epoch: 4, PLCC: 0.9547, SROCC: 0.9471, MSE: 0.0043
Validation Results - Split: 0, Epo

KeyboardInterrupt: 

: 

In [2]:
exp_name = "CLIP_image-(ResNet101)-0.0_noB-AIGC"

class FusionModel(nn.Module):
    def __init__(self):
        super(FusionModel, self).__init__()
        
        # Load the pretrained ResNet18 model
        self.quality_model = MetaIQA('model_IQA/TID2013_KADID10K_IQA_Meta_resnet18.pt')
        input_size = 1024
        intermediate_size = 512

        self.fc_clip_feat = nn.Linear(512, 512)
        self.fc_clip_feat.weight.data.normal_(0, 0.0)
        self.fc_clip_feat.bias.data.zero_()
        # self.bn_clip_feat = nn.BatchNorm1d(512)
        self.relu_clip_feat = nn.PReLU()
        self.drop_clip_feat = nn.Dropout(p=0.5)
        self.fc_final = nn.Linear(512, 1)
        
        # # Define the fully connected layer
        self.fc1 = nn.Linear(input_size, input_size)
        # # self.bn1 = nn.BatchNorm1d(input_size)
        # # self.relu1 = nn.PReLU()
        # # self.drop1 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(input_size, intermediate_size)
        # # self.bn2 = nn.BatchNorm1d(intermediate_size)
        # # self.relu2 = nn.PReLU()
        # # self.drop2 = nn.Dropout(p=0.5)
        self.fc3 = nn.Linear(intermediate_size, 1)
        self.sig = nn.Sigmoid()

    
        
    def forward(self, x, similarity_features):
        # Pass the input through the pretrained model 
        quality, quality_features = self.quality_model(x)
        # print (quality_features.shape)

        similarity_features = self.fc_clip_feat(similarity_features)
        # similarity_features = self.bn_clip_feat(similarity_features)
        similarity_features = self.relu_clip_feat(similarity_features)
        similarity_features = self.drop_clip_feat(similarity_features)

        final = self.fc_final(similarity_features)
        # Concatenate the final prediction output and the new input
        combined = torch.cat((quality_features, final), dim=1)
        # x = quality_features
        # print(x.shape)
        
        # # Pass the concatenated tensor through the fully connected layer
        # x = self.fc1(combined)
        # # x = self.bn1(x)
        # # x = self.relu1(x)
        # # x = self.drop1(x)
        # x = self.fc2(x)
        # # x = self.bn2(x)
        # # x = self.relu2(x)
        # # x = self.drop2(x)
        # x = self.fc3(x)
        return self.sig(final)
        # return x
    
def fetch_example(data):
    inputs = data['image']
    similarity_features = data['clip_image_features'] #- data['clip_text_features']
    batch_size = inputs.size()[0]
    labels = data['rating'].view(batch_size, -1) / 5.0
    
    inputs, similarity_features, labels = Variable(inputs.float().cuda()), Variable(similarity_features.float().cuda()), Variable(labels.float().cuda())
    return [inputs, similarity_features], labels

finetune_model(FusionModel, fetch_example, exp_name, num_epochs = num_epochs, num_splits = num_splits)

decay_rate is set to 1.0
Train Results - Split: 0, Epoch: 0, PLCC: 0.7700, SROCC: 0.7075, MSE: 0.0255
Validation Results - Split: 0, Epoch: 0, PLCC: 0.7396, SROCC: 0.7346, MSE: 0.0246, Best PLCC: 0.7396, Best SROCC: 0.7346, Best MSE: 0.0246
Train Results - Split: 0, Epoch: 1, PLCC: 0.7787, SROCC: 0.7184, MSE: 0.0214
Validation Results - Split: 0, Epoch: 1, PLCC: 0.7512, SROCC: 0.7505, MSE: 0.0207, Best PLCC: 0.7512, Best SROCC: 0.7505, Best MSE: 0.0207
Train Results - Split: 0, Epoch: 2, PLCC: 0.7902, SROCC: 0.7333, MSE: 0.0178
Validation Results - Split: 0, Epoch: 2, PLCC: 0.7637, SROCC: 0.7638, MSE: 0.0173, Best PLCC: 0.7637, Best SROCC: 0.7638, Best MSE: 0.0173
Train Results - Split: 0, Epoch: 3, PLCC: 0.8035, SROCC: 0.7496, MSE: 0.0149
Validation Results - Split: 0, Epoch: 3, PLCC: 0.7774, SROCC: 0.7785, MSE: 0.0146, Best PLCC: 0.7774, Best SROCC: 0.7785, Best MSE: 0.0146
Train Results - Split: 0, Epoch: 4, PLCC: 0.8167, SROCC: 0.7667, MSE: 0.0127
Validation Results - Split: 0, Epo

: 

In [2]:
exp_name = "CLIP_image-(vit-14-768)-0.0_noB"

class FusionModel(nn.Module):
    def __init__(self):
        super(FusionModel, self).__init__()
        
        # Load the pretrained ResNet18 model
        self.quality_model = MetaIQA('model_IQA/TID2013_KADID10K_IQA_Meta_resnet18.pt')
        input_size = 1024
        intermediate_size = 512

        self.fc = nn.Linear(768, 768)
        self.fc.weight.data.normal_(0, 0.0)
        self.fc.bias.data.zero_()
        # self.bn_clip_feat = nn.BatchNorm1d(512)
        self.relu = nn.PReLU()
        self.drop = nn.Dropout(p=0.5)
        self.sig = nn.Sigmoid()

        
        # Define the fully connected layer
        self.fc1 = nn.Linear(input_size, input_size)
        self.bn1 = nn.BatchNorm1d(input_size)
        self.relu1 = nn.PReLU()
        self.drop1 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(input_size, intermediate_size)
        self.bn2 = nn.BatchNorm1d(intermediate_size)
        self.relu2 = nn.PReLU()
        self.drop2 = nn.Dropout(p=0.5)
        self.fc3 = nn.Linear(intermediate_size, 1)
        # self.sig = nn.Sigmoid()

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # Weight initialization reference: https://arxiv.org/abs/1502.01852
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
        
    def forward(self, x, similarity_features):
        # Pass the input through the pretrained model 
        # quality, quality_features = self.quality_model(x)

        # similarity_features = self.fc_clip_feat(similarity_features)
        # # # similarity_features = self.bn_clip_feat(similarity_features)
        # similarity_features = self.relu_clip_feat(similarity_features)
        # similarity_features = self.drop_clip_feat(similarity_features)
        # Concatenate the final prediction output and the new input
        # x = torch.cat((quality_features, similarity_features), dim=1)
        x = similarity_features
        
        x = self.fc(x)
        # x = self.bn1(x)
        x = self.relu(x)
        x = self.drop(x)
        # x = self.fc2(x)
        # x = self.bn2(x)
        # x = self.relu2(x)
        # x = self.drop2(x)
        # x = self.fc3(x)
        return self.sig(x)
        # return x
    
def fetch_example(data):
    inputs = data['image']
    similarity_features = data['clip_image_features'] #- data['clip_text_features']
    batch_size = inputs.size()[0]
    labels = data['rating'].view(batch_size, -1) / 5.0
    
    inputs, similarity_features, labels = Variable(inputs.float().cuda()), Variable(similarity_features.float().cuda()), Variable(labels.float().cuda())
    return [inputs, similarity_features], labels

finetune_model(FusionModel, fetch_example, exp_name, num_epochs = num_epochs, num_splits = num_splits)

decay_rate is set to 1.0
Train Results - Split: 0, Epoch: 0, PLCC: 0.7815, SROCC: 0.7478, MSE: 0.0269
Validation Results - Split: 0, Epoch: 0, PLCC: 0.7353, SROCC: 0.7644, MSE: 0.0260, Best PLCC: 0.7353, Best SROCC: 0.7644, Best MSE: 0.0260
Train Results - Split: 0, Epoch: 1, PLCC: 0.7978, SROCC: 0.7788, MSE: 0.0254
Validation Results - Split: 0, Epoch: 1, PLCC: 0.7606, SROCC: 0.7901, MSE: 0.0246, Best PLCC: 0.7606, Best SROCC: 0.7901, Best MSE: 0.0246
Train Results - Split: 0, Epoch: 2, PLCC: 0.8028, SROCC: 0.8046, MSE: 0.0244
Validation Results - Split: 0, Epoch: 2, PLCC: 0.7660, SROCC: 0.8026, MSE: 0.0238, Best PLCC: 0.7660, Best SROCC: 0.8026, Best MSE: 0.0238
Train Results - Split: 0, Epoch: 3, PLCC: 0.8050, SROCC: 0.8151, MSE: 0.0236
Validation Results - Split: 0, Epoch: 3, PLCC: 0.7600, SROCC: 0.7980, MSE: 0.0231, Best PLCC: 0.7660, Best SROCC: 0.8026, Best MSE: 0.0231
Train Results - Split: 0, Epoch: 4, PLCC: 0.8081, SROCC: 0.8183, MSE: 0.0229
Validation Results - Split: 0, Epo

: 

In [35]:
exp_name = "no-similarity-k20"

class FusionModel(nn.Module):
    def __init__(self):
        super(FusionModel, self).__init__()
        
        # Load the pretrained ResNet18 model
        self.quality_model = MetaIQA('model_IQA/TID2013_KADID10K_IQA_Meta_resnet18.pt')
        self.sig = nn.Sigmoid()

        
    def forward(self, x, similarity_features):
        # Pass the input through the pretrained model 
        quality, quality_features = self.quality_model(x)
        return self.sig(quality_features)
        # return x
    
def fetch_example(data):
    inputs = data['image']
    similarity_features = data['clip_image_features'] - data['clip_text_features']
    batch_size = inputs.size()[0]
    labels = data['rating'].view(batch_size, -1) / 5.0
    
    inputs, similarity_features, labels = Variable(inputs.float().cuda()), Variable(similarity_features.float().cuda()), Variable(labels.float().cuda())
    return [inputs, similarity_features], labels

finetune_model(FusionModel, fetch_example, exp_name, num_epochs = num_epochs, num_splits = num_splits)

decay_rate is set to 1.0
Train Results - Fold: 0, Epoch: 0, PLCC: 0.7930, SROCC: 0.6967, MSE: 0.0199
Validation Results - Fold: 0, Epoch: 0, PLCC: 0.6939, SROCC: 0.7413, MSE: 0.0302, Best PLCC: 0.6939, Best SROCC: 0.7413, Best MSE: 0.0302
Train Results - Fold: 0, Epoch: 1, PLCC: 0.8714, SROCC: 0.8190, MSE: 0.0194
Validation Results - Fold: 0, Epoch: 1, PLCC: 0.8637, SROCC: 0.8323, MSE: 0.0281, Best PLCC: 0.8637, Best SROCC: 0.8323, Best MSE: 0.0281
Train Results - Fold: 0, Epoch: 2, PLCC: 0.9056, SROCC: 0.8680, MSE: 0.0183
Validation Results - Fold: 0, Epoch: 2, PLCC: 0.8903, SROCC: 0.8372, MSE: 0.0267, Best PLCC: 0.8903, Best SROCC: 0.8372, Best MSE: 0.0267
Train Results - Fold: 0, Epoch: 3, PLCC: 0.9174, SROCC: 0.9096, MSE: 0.0178
Validation Results - Fold: 0, Epoch: 3, PLCC: 0.8680, SROCC: 0.8219, MSE: 0.0266, Best PLCC: 0.8903, Best SROCC: 0.8372, Best MSE: 0.0266
Train Results - Fold: 0, Epoch: 4, PLCC: 0.8923, SROCC: 0.9241, MSE: 0.0178
Validation Results - Fold: 0, Epoch: 4, PLC

In [36]:
exp_name = "intermediate-fusion-small-init-similarity-1.0"

class FusionModel(nn.Module):
    def __init__(self):
        super(FusionModel, self).__init__()
        
        # Load the pretrained ResNet18 model
        self.quality_model = MetaIQA('model_IQA/TID2013_KADID10K_IQA_Meta_resnet18.pt')
        input_size = 1024
        intermediate_size = 512

        self.fc_clip_feat = nn.Linear(512, 512)
        self.fc_clip_feat.weight.data.normal_(0, 1.0)
        self.fc_clip_feat.bias.data.zero_()
        self.bn_clip_feat = nn.BatchNorm1d(512)
        self.relu_clip_feat = nn.PReLU()
        self.drop_clip_feat = nn.Dropout(p=0.5)
        
        # Define the fully connected layer
        self.fc1 = nn.Linear(input_size, input_size)
        self.bn1 = nn.BatchNorm1d(input_size)
        self.relu1 = nn.PReLU()
        self.drop1 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(input_size, intermediate_size)
        self.bn2 = nn.BatchNorm1d(intermediate_size)
        self.relu2 = nn.PReLU()
        self.drop2 = nn.Dropout(p=0.5)
        self.fc3 = nn.Linear(intermediate_size, 1)
        self.sig = nn.Sigmoid()
        
    def forward(self, x, similarity_features):
        # Pass the input through the pretrained model 
        quality, quality_features = self.quality_model(x)

        similarity_features = self.fc_clip_feat(similarity_features)
        similarity_features = self.bn_clip_feat(similarity_features)
        similarity_features = self.relu_clip_feat(similarity_features)
        similarity_features = self.drop_clip_feat(similarity_features)
        # Concatenate the final prediction output and the new input
        x = torch.cat((quality_features, similarity_features), dim=1)
        
        # Pass the concatenated tensor through the fully connected layer
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.drop1(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.drop2(x)
        x = self.fc3(x)
        return self.sig(x)
        # return x
    
def fetch_example(data):
    inputs = data['image']
    similarity_features = data['clip_image_features'] - data['clip_text_features']
    batch_size = inputs.size()[0]
    labels = data['rating'].view(batch_size, -1) / 5.0
    
    inputs, similarity_features, labels = Variable(inputs.float().cuda()), Variable(similarity_features.float().cuda()), Variable(labels.float().cuda())
    return [inputs, similarity_features], labels

finetune_model(FusionModel, fetch_example, exp_name, num_epochs = num_epochs, num_splits = num_splits)

decay_rate is set to 1.0
Train Results - Fold: 0, Epoch: 0, PLCC: 0.8586, SROCC: 0.8138, MSE: 0.0080
Validation Results - Fold: 0, Epoch: 0, PLCC: 0.8741, SROCC: 0.8391, MSE: 0.0095, Best PLCC: 0.8741, Best SROCC: 0.8391, Best MSE: 0.0095
Train Results - Fold: 0, Epoch: 1, PLCC: 0.8920, SROCC: 0.8553, MSE: 0.0074
Validation Results - Fold: 0, Epoch: 1, PLCC: 0.8819, SROCC: 0.8738, MSE: 0.0096, Best PLCC: 0.8819, Best SROCC: 0.8738, Best MSE: 0.0095
Train Results - Fold: 0, Epoch: 2, PLCC: 0.9134, SROCC: 0.8816, MSE: 0.0053
Validation Results - Fold: 0, Epoch: 2, PLCC: 0.8960, SROCC: 0.8741, MSE: 0.0079, Best PLCC: 0.8960, Best SROCC: 0.8741, Best MSE: 0.0079
Train Results - Fold: 0, Epoch: 3, PLCC: 0.9306, SROCC: 0.9085, MSE: 0.0050
Validation Results - Fold: 0, Epoch: 3, PLCC: 0.8959, SROCC: 0.8812, MSE: 0.0084, Best PLCC: 0.8960, Best SROCC: 0.8812, Best MSE: 0.0079
Train Results - Fold: 0, Epoch: 4, PLCC: 0.9377, SROCC: 0.9178, MSE: 0.0047
Validation Results - Fold: 0, Epoch: 4, PLC

In [37]:
exp_name = "intermediate-fusion-small-init-no-similarity"

class FusionModel(nn.Module):
    def __init__(self):
        super(FusionModel, self).__init__()
        
        # Load the pretrained ResNet18 model
        self.quality_model = MetaIQA('model_IQA/TID2013_KADID10K_IQA_Meta_resnet18.pt')
        input_size = 1024
        intermediate_size = 512

        # self.fc_clip_feat = nn.Linear(512, 512)
        # self.fc_clip_feat.weight.data.normal_(0, 0.0008)
        # self.fc_clip_feat.bias.data.zero_()
        # self.bn_clip_feat = nn.BatchNorm1d(512)
        # self.relu_clip_feat = nn.PReLU()
        # self.drop_clip_feat = nn.Dropout(p=0.5)
        
        # Define the fully connected layer
        self.fc1 = nn.Linear(512, input_size)
        self.bn1 = nn.BatchNorm1d(input_size)
        self.relu1 = nn.PReLU()
        self.drop1 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(input_size, intermediate_size)
        self.bn2 = nn.BatchNorm1d(intermediate_size)
        self.relu2 = nn.PReLU()
        self.drop2 = nn.Dropout(p=0.5)
        self.fc3 = nn.Linear(intermediate_size, 1)
        self.sig = nn.Sigmoid()

        # for m in self.modules():
        #     if isinstance(m, nn.Conv2d):
        #         # Weight initialization reference: https://arxiv.org/abs/1502.01852
        #         n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
        #         m.weight.data.normal_(0, math.sqrt(2. / n))
        #     elif isinstance(m, nn.BatchNorm2d):
        #         m.weight.data.fill_(1)
        #         m.bias.data.zero_()
            # elif isinstance(m, nn.Linear):
            #     m.weight.data.normal_(0, 0.02)
            #     m.bias.data.zero_()
        
    def forward(self, x, similarity_features):
        # Pass the input through the pretrained model 
        quality, quality_features = self.quality_model(x)

        # similarity_features = self.fc_clip_feat(similarity_features)
        # similarity_features = self.bn_clip_feat(similarity_features)
        # similarity_features = self.relu_clip_feat(similarity_features)
        # similarity_features = self.drop_clip_feat(similarity_features)
        # Concatenate the final prediction output and the new input
        # x = torch.cat((quality_features, similarity_features), dim=1)
        x = quality_features

        # Pass the concatenated tensor through the fully connected layer
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.drop1(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.drop2(x)
        x = self.fc3(x)
        return self.sig(x)
        # return x
    
def fetch_example(data):
    inputs = data['image']
    similarity_features = data['clip_image_features'] - data['clip_text_features']
    batch_size = inputs.size()[0]
    labels = data['rating'].view(batch_size, -1) / 5.0
    
    inputs, similarity_features, labels = Variable(inputs.float().cuda()), Variable(similarity_features.float().cuda()), Variable(labels.float().cuda())
    return [inputs, similarity_features], labels

finetune_model(FusionModel, fetch_example, exp_name, num_epochs = num_epochs, num_splits = num_splits)

decay_rate is set to 1.0
Train Results - Fold: 0, Epoch: 0, PLCC: 0.8365, SROCC: 0.7709, MSE: 0.0097
Validation Results - Fold: 0, Epoch: 0, PLCC: 0.8188, SROCC: 0.7556, MSE: 0.0149, Best PLCC: 0.8188, Best SROCC: 0.7556, Best MSE: 0.0149
Train Results - Fold: 0, Epoch: 1, PLCC: 0.8744, SROCC: 0.8454, MSE: 0.0071
Validation Results - Fold: 0, Epoch: 1, PLCC: 0.8532, SROCC: 0.8241, MSE: 0.0107, Best PLCC: 0.8532, Best SROCC: 0.8241, Best MSE: 0.0107
Train Results - Fold: 0, Epoch: 2, PLCC: 0.8864, SROCC: 0.8607, MSE: 0.0067
Validation Results - Fold: 0, Epoch: 2, PLCC: 0.8574, SROCC: 0.8460, MSE: 0.0104, Best PLCC: 0.8574, Best SROCC: 0.8460, Best MSE: 0.0104
Train Results - Fold: 0, Epoch: 3, PLCC: 0.9044, SROCC: 0.8887, MSE: 0.0064
Validation Results - Fold: 0, Epoch: 3, PLCC: 0.8687, SROCC: 0.8511, MSE: 0.0096, Best PLCC: 0.8687, Best SROCC: 0.8511, Best MSE: 0.0096
Train Results - Fold: 0, Epoch: 4, PLCC: 0.9081, SROCC: 0.9027, MSE: 0.0053
Validation Results - Fold: 0, Epoch: 4, PLC

In [2]:
exp_name = "CLIP_FEATURES-IQA-ViT-14-wb-noinit"

class FusionModel(nn.Module):
    def __init__(self):
        super(FusionModel, self).__init__()
        
        # Load the pretrained ResNet18 model
        self.quality_model = MetaIQA('model_IQA/TID2013_KADID10K_IQA_Meta_resnet18.pt')
        input_size = 1040
        intermediate_size = 512

        self.fc_clip_feat = nn.Linear(512, 512)
        # self.fc_clip_feat.weight.data.normal_(0, 0.01)
        # self.fc_clip_feat.bias.data.zero_()
        self.bn_clip_feat = nn.BatchNorm1d(768)
        self.relu_clip_feat = nn.PReLU()
        self.drop_clip_feat = nn.Dropout(p=0.5)
        
        # Define the fully connected layer
        self.fc1 = nn.Linear(input_size, input_size)
        self.bn1 = nn.BatchNorm1d(input_size)
        self.relu1 = nn.PReLU()
        self.drop1 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(input_size, intermediate_size)
        self.bn2 = nn.BatchNorm1d(intermediate_size)
        self.relu2 = nn.PReLU()
        self.drop2 = nn.Dropout(p=0.5)
        self.fc3 = nn.Linear(intermediate_size, 1)
        self.sig = nn.Sigmoid()

        
    def forward(self, x, similarity_features):
        # Pass the input through the pretrained model 
        quality, quality_features = self.quality_model(x)

        similarity_features = self.fc_clip_feat(similarity_features)
        similarity_features = self.bn_clip_feat(similarity_features)
        similarity_features = self.relu_clip_feat(similarity_features)
        similarity_features = self.drop_clip_feat(similarity_features)
        # Concatenate the final prediction output and the new input
        x = torch.cat((quality_features, similarity_features), dim=1)
        
        # Pass the concatenated tensor through the fully connected layer
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.drop1(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.drop2(x)
        x = self.fc3(x)
        return self.sig(x)
        # return x
    
def fetch_example(data):
    inputs = data['image']
    similarity_features = data['clip_image_features'] - data['clip_text_features']
    batch_size = inputs.size()[0]
    labels = data['rating'].view(batch_size, -1) / 5.0
    
    inputs, similarity_features, labels = Variable(inputs.float().cuda()), Variable(similarity_features.float().cuda()), Variable(labels.float().cuda())
    return [inputs, similarity_features], labels

finetune_model(FusionModel, fetch_example, exp_name, num_epochs = num_epochs, num_splits = num_splits)

decay_rate is set to 1.0
Train Results - Split: 0, Epoch: 0, PLCC: 0.8608, SROCC: 0.8052, MSE: 0.0095
Validation Results - Split: 0, Epoch: 0, PLCC: 0.8095, SROCC: 0.7727, MSE: 0.0110, Best PLCC: 0.8095, Best SROCC: 0.7727, Best MSE: 0.0110
Train Results - Split: 0, Epoch: 1, PLCC: 0.8912, SROCC: 0.8486, MSE: 0.0065
Validation Results - Split: 0, Epoch: 1, PLCC: 0.8242, SROCC: 0.7813, MSE: 0.0093, Best PLCC: 0.8242, Best SROCC: 0.7813, Best MSE: 0.0093
Train Results - Split: 0, Epoch: 2, PLCC: 0.9087, SROCC: 0.8762, MSE: 0.0056
Validation Results - Split: 0, Epoch: 2, PLCC: 0.8399, SROCC: 0.8140, MSE: 0.0084, Best PLCC: 0.8399, Best SROCC: 0.8140, Best MSE: 0.0084
Train Results - Split: 0, Epoch: 3, PLCC: 0.9164, SROCC: 0.8876, MSE: 0.0052
Validation Results - Split: 0, Epoch: 3, PLCC: 0.8176, SROCC: 0.7716, MSE: 0.0103, Best PLCC: 0.8399, Best SROCC: 0.8140, Best MSE: 0.0084
Train Results - Split: 0, Epoch: 4, PLCC: 0.9328, SROCC: 0.9142, MSE: 0.0041
Validation Results - Split: 0, Epo

: 

In [ ]:
exp_name = "CLIP_FEATURES-IQA-ViT-14-wb-noinit"

class FusionModel(nn.Module):
    def __init__(self):
        super(FusionModel, self).__init__()
        
        # Load the pretrained ResNet18 model
        self.quality_model = MetaIQA('model_IQA/TID2013_KADID10K_IQA_Meta_resnet18.pt')
        input_size = 1040
        intermediate_size = 512

        self.fc_clip_feat = nn.Linear(512, 512)
        # self.fc_clip_feat.weight.data.normal_(0, 0.01)
        # self.fc_clip_feat.bias.data.zero_()
        self.bn_clip_feat = nn.BatchNorm1d(768)
        self.relu_clip_feat = nn.PReLU()
        self.drop_clip_feat = nn.Dropout(p=0.5)
        
        # Define the fully connected layer
        self.fc1 = nn.Linear(input_size, input_size)
        self.bn1 = nn.BatchNorm1d(input_size)
        self.relu1 = nn.PReLU()
        self.drop1 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(input_size, intermediate_size)
        self.bn2 = nn.BatchNorm1d(intermediate_size)
        self.relu2 = nn.PReLU()
        self.drop2 = nn.Dropout(p=0.5)
        self.fc3 = nn.Linear(intermediate_size, 1)
        self.sig = nn.Sigmoid()

        
    def forward(self, x, similarity_features):
        # Pass the input through the pretrained model 
        quality, quality_features = self.quality_model(x)

        # similarity_features = self.fc_clip_feat(similarity_features)
        # similarity_features = self.bn_clip_feat(similarity_features)
        # similarity_features = self.relu_clip_feat(similarity_features)
        # similarity_features = self.drop_clip_feat(similarity_features)
        # Concatenate the final prediction output and the new input
        x = torch.cat((quality_features, similarity_features), dim=1)
        
        # Pass the concatenated tensor through the fully connected layer
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.drop1(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.drop2(x)
        x = self.fc3(x)
        return self.sig(x)
        # return x
    
def fetch_example(data):
    inputs = data['image']
    similarity_features = data['clip_image_features'] - data['clip_text_features']
    batch_size = inputs.size()[0]
    labels = data['rating'].view(batch_size, -1) / 5.0
    
    inputs, similarity_features, labels = Variable(inputs.float().cuda()), Variable(similarity_features.float().cuda()), Variable(labels.float().cuda())
    return [inputs, similarity_features], labels

finetune_model(FusionModel, fetch_example, exp_name, num_epochs = num_epochs, num_splits = num_splits)